In [1]:
import numpy as np
import matplotlib.pyplot as plt

import joblib

import random
from deap import base
from deap import creator
from deap import tools

from deap import algorithms
from time import time
import multiprocessing

import array
import random
import json

import os

import pandas as pd

In [2]:
matrix_np = joblib.load('matrix_np.pkl')

In [3]:
#reduced_matrix_np = joblib.load('reduced_matrix_np.pkl')

# Parameters

In [4]:
matrix_algorithm = matrix_np
NUM_CLUSTERS = 7

# Distance to cluster function

In [5]:
def auxiliarFunction(documents, dimensions, number_topics):

    counter_docs = [0 for x in range(0,number_topics)]
    
    distance_to_cluster_2 = [0 for x in range(0,number_topics)] 
    
    for document in documents: 
        distance_to_cluster = []   
        counter = 0
        for dim in range(0,number_topics): 
            distance_to_cluster.append(np.linalg.norm(document-dimensions[counter:counter+7]))
            counter = counter + 7
            
        best_distance_index = distance_to_cluster.index(min(distance_to_cluster)) 
        
        counter_docs[best_distance_index] += 1
    
        distance_to_cluster_2[best_distance_index] += pow(distance_to_cluster[best_distance_index], 2)
        
     
    return counter_docs, distance_to_cluster_2

In [6]:
def fobjQuadraticError(k):

    num_topics = int((len(k))/7)
    
    # 1º Number of documents to each cluster
    counter_documents, euclidean_dist_2 = auxiliarFunction(documents = matrix_algorithm, dimensions= k, number_topics = num_topics)
    
    bool_check = 0 in counter_documents    

   # 2º Fitness fuction
    if(bool_check):
        total_value = 5 # We give a high value to those clusters who do not have documents
        
    else:
        value1 = 0
        for i in range(0,num_topics): 
            value2 = euclidean_dist_2[i] 
            value1 += ((1/counter_documents[i])*value2)

        total_value = (value1 / num_topics) 

    return (total_value),

# Genetic Algorithm

## Folders creation

In [7]:
path = "./results/fobjQuadraticError"
try:
    if not os.path.isdir(path):
        os.makedirs(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory ./results/fobjQuadraticError 


## Genetic operators

### Initial Population

In [8]:
def load_individuals(creator,n):
    individuals = []
    ones = [0 for x in range(0,49)]
    j=0
    for i in range(49):
        if (i % 7 == 0):
            ones[i+j] = 1
            j = j+1
            
    individual = creator(ones)
    individuals.append(individual)
    
    for l in range(int(n*0.1)-1):
        one = [0 for x in range(0,49)]
        m1 = [0,1,2,3,4,5,6]
        random.shuffle(m1)
        for j in range (7):
            for i in range(7):
                one[7*j+i] = ones[7*m1[j]+i]
        individual = creator(one)
        individuals.append(individual)
        
    
    Rfactor = [0 for x in range(0,3)]
    for i in range(3):
        Rfactor[i] = 1/4*(i+1)
    
    for l in range(int(n*0.9/3)):
        two = [0 for x in range(0,49)]
        ind1 = individuals[random.randrange(0, int(n*0.1))]
        ind2 = individuals[random.randrange(0, int(n*0.1))]
        for t in range(3):
            two = [0 for x in range(0,49)]
            for j in range(7):
                for i in range(7):
                    two[7*j+i] = ind1[7*j+i]*Rfactor[t]+ind2[7*j+i]*(1-Rfactor[t])
                
            individual = creator(two)
            individuals.append(individual)
        
    return individuals

### Mutation operators

#### Global mutation operator

In [9]:
def customMutate(ind,b_ind,g,m_prob):
    t = random.randrange(0, 10)
    
    if t>4 and g >1:
        new_clusters = redist2(b_ind)
        ind[0:49] = new_clusters[0:49]
    else: #first mutation operator
        for j in range (7):
            if random.random() < m_prob:
                m1 = random.randrange(0, 7)
                m2 = random.randrange(0, 7)
                auxNum = ind[7*j+m1]
                ind[7*j+m1] = ind[7*j+m2]
                ind[7*j+m2] = auxNum        
        
    return ind,

In [10]:
def isone(k):
    sumat = 0
    for i in range(7):
        sumat = sumat + k[i]
    return sumat

In [11]:
def checkbounds(k):
    for i in range(7):
        if k[i]>1:
            k[i]=1
        if k[i]<0:
            k[i]=0
    return k

In [12]:
def redist(b_ind):
    documents = matrix_algorithm 
    new_clusters = [0 for x in range(0,49)]
    counter = 0
    for dim in range(0,7): 
        min_dist_doc = [0 for x in range(0,7)]
        max_dist_doc = [0 for x in range(0,7)]
        min_dist = 100000000
        max_dist = 0
        for document in documents: 
                dist = np.linalg.norm(document-b_ind[counter:counter+7])
                if dist<min_dist:
                    min_dist = dist
                    min_dist_doc = document
                if dist>max_dist:
                    max_dist = dist
                    max_dist_doc = document
                    
        new_clusters[counter:counter+7] = (min_dist_doc+max_dist_doc)/2
        counter = counter + 7
                 
    return new_clusters

#### 2nd Mutation Operator

In [13]:
def redist2(b_ind):
    documents = matrix_algorithm 
    new_clusters = [0 for x in range(0,49)]
    diff = [0 for x in range(0,49)]
    counter = 0
    Rf = random.random()
    for dim in range(0,7): 
        min_dist_doc = [0 for x in range(0,7)]
        max_dist_doc = [0 for x in range(0,7)]
        
        min_dist = 100000000
        max_dist = 0
        for document in documents: 
                dist = np.linalg.norm(document-b_ind[counter:counter+7])
                if dist<min_dist:
                    min_dist = dist
                    min_dist_doc = document
                if dist>max_dist:
                    max_dist = dist
                    max_dist_doc = document
 

        diff[counter:counter+7] = (min_dist_doc+max_dist_doc)/2 - b_ind[counter:counter+7] 

        for j in range(counter,counter+7):
            new_clusters[j] = b_ind[j]+Rf*diff[j]
        
        sumat = isone(new_clusters[counter:counter+7])    
        while sumat<0.99999999999999 or sumat>1.0000000000000:
            new_clusters[counter:counter+7] = new_clusters[counter:counter+7]+ (1-sumat)/7
            new_clusters = checkbounds(new_clusters)
            sumat = isone(new_clusters[counter:counter+7]) 
            
        counter = counter + 7
        
    return new_clusters

### Define evaluation function

In [14]:
def evalfobj(individual):
    return fobjQuadraticError(individual)

### Crossover operator

In [15]:
def customMate(ind1, ind2):
    aux1 = ind1
    aux2 = ind2
      
    Rfactor = [0 for x in range(0,2)]
    for i in range(2):
        Rfactor[i] = random.random()
    for i in range(49):
        ind1[i] = aux1[i]*Rfactor[0]+aux2[i]*(1-Rfactor[0])
        ind2[i] = aux1[i]*Rfactor[1]+aux2[i]*(1-Rfactor[1])

    return (ind1,ind2)

## Genetic process and save results 

In [16]:
if __name__ == "__main__":
    
    CLUSTER_DIM = 7 * NUM_CLUSTERS

    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    
    toolbox.register("population", load_individuals,  creator.Individual)


    #toolbox.register("attr_flt", random.uniform, 0, 1)
    #toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_flt,n=CLUSTER_DIM)

    #toolbox.register("population", tools.initRepeat, list, toolbox.individual)


    toolbox.register("mate", customMate)
    toolbox.register("mutate", customMutate)

    toolbox.register("select", tools.selTournament, tournsize=10)
    
    toolbox.register("evaluate", evalfobj)
    
    pool = multiprocessing.Pool(processes = 4)
    toolbox.register('map', pool.map)
    
    numberInd = 100
    
    # MAIN GENETIC ALGORITHM
    random.seed(169)
    min_iterations = []
    max_iterations = []
    avg_iterations = []
    std_iterations = []
    ind_iterations = []
    
    CXPB, MUTPB = 0.8, 0.35
    m_prob = 0.35
    
    pop = toolbox.population(n=100)
    fitnesses = list(toolbox.map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit 
        
    fits = [ind.fitness.values[0] for ind in pop] 
    best_indiv = []
    g = 0
    # Begin the evolution
    while g < 10000:

       # start_time = time()
    
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)

        offspring = toolbox.select(pop, len(pop))
    
        offspring = list(map(toolbox.clone, offspring))
        

        for child1, child2 in zip(offspring[::2], offspring[1::2]): 
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values 
                del child2.fitness.values 
                
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant,best_indiv,g,m_prob)
                del mutant.fitness.values  

        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
            
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            

        pop[:] = offspring

        
        fits = [ind.fitness.values[0] for ind in pop]
        
        fit_maxi = 100000000000000000000
        individ = []
        for ind in pop:
            if (ind.fitness.values[0]<fit_maxi):
                fit_maxi = ind.fitness.values[0]
                individ = ind
        best_indiv = individ

            
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        print("  Min %s" % min(fits))
        
  
        
        min_iterations.append(min(fits))
        max_iterations.append(max(fits))
        avg_iterations.append(mean)
        std_iterations.append(std)
        ind_iterations.append(individ)
        
        if (g % 100 == 0):
            print(best_indiv)
            path = "./results/fobjQuadraticError/iteration{}".format(g)
            try:
                if not os.path.isdir(path):
                    os.makedirs(path)
            except OSError:
                print ("Creation of the directory %s failed" % path)
            else:
                print ("Successfully created the directory %s " % path)
        
            print("-----------------------------------------------------------------------------------------")
            print("Guardamos valores")
            joblib.dump(min_iterations, "results/fobjQuadraticError/iteration{}/best_gen_fitness_iter{}.pkl".format(g, g))
            joblib.dump(max_iterations, "results/fobjQuadraticError/iteration{}/worst_gen_fitness_iter{}.pkl".format(g, g))
            joblib.dump(avg_iterations, "results/fobjQuadraticError/iteration{}/mean_gen_fitness_iter{}.pkl".format(g, g))
            joblib.dump(std_iterations, "results/fobjQuadraticError/iteration{}/std_gen_fitness_iter{}.pkl".format(g, g))
            joblib.dump(ind_iterations, "results/fobjQuadraticError/iteration{}/best_gen_individuals_iter{}.pkl".format(g, g))
            joblib.dump(pop[:], "results/fobjQuadraticError/iteration{}/last_pop{}.pkl".format(g, g))

            print('-----------------------------------------------------------------------------------------')

       # print("--- {} s seconds to perform iteration number: {} ---".format((time() - start_time), g))
            
    #best = pop[np.argmin([toolbox.evaluate(x) for x in pop])]
    final_pop = pop[:]
    
    joblib.dump(min_iterations, "results/fobjQuadraticError/best_gen_fitness_iter{}.pkl".format(10000))
    joblib.dump(max_iterations, "results/fobjQuadraticError/worst_gen_fitness_iter{}.pkl".format(10000))
    joblib.dump(avg_iterations, "results/fobjQuadraticError/mean_gen_fitness_iter{}.pkl".format(10000))
    joblib.dump(std_iterations, "results/fobjQuadraticError/std_gen_fitness_iter{}.pkl".format(10000))
    joblib.dump(ind_iterations, "results/fobjQuadraticError/best_gen_individuals_iter{}.pkl".format(10000))
    joblib.dump(final_pop, "results/fobjQuadraticError/final_pop_iter{}.pkl".format(10000))


-- Generation 1 --
  Min 0.1341095635165228
-- Generation 2 --
  Min 0.12048518067937755
-- Generation 3 --
  Min 0.11085903357533847
-- Generation 4 --
  Min 0.10555042035051167
-- Generation 5 --
  Min 0.10183861712967444
-- Generation 6 --
  Min 0.09900364821128137
-- Generation 7 --
  Min 0.09794333395747683
-- Generation 8 --
  Min 0.0975813916602891
-- Generation 9 --
  Min 0.0968751400218489
-- Generation 10 --
  Min 0.09675036906330327
-- Generation 11 --
  Min 0.09596258452680614
-- Generation 12 --
  Min 0.09490658435428834
-- Generation 13 --
  Min 0.09429536802129825
-- Generation 14 --
  Min 0.09407819796897572
-- Generation 15 --
  Min 0.0937058064444
-- Generation 16 --
  Min 0.09257944047514122
-- Generation 17 --
  Min 0.09238734466514349
-- Generation 18 --
  Min 0.09235172498729931
-- Generation 19 --
  Min 0.09211236168933097
-- Generation 20 --
  Min 0.09198045745633607
-- Generation 21 --
  Min 0.09195670947322847
-- Generation 22 --
  Min 0.09184954091625178
-- G

  Min 0.08991382310123273
-- Generation 151 --
  Min 0.08991382310123273
-- Generation 152 --
  Min 0.08991382310123273
-- Generation 153 --
  Min 0.08991382310123273
-- Generation 154 --
  Min 0.08991382310123273
-- Generation 155 --
  Min 0.08991382310123273
-- Generation 156 --
  Min 0.08991382310123273
-- Generation 157 --
  Min 0.08991382310123273
-- Generation 158 --
  Min 0.08991382310123273
-- Generation 159 --
  Min 0.08991382310123273
-- Generation 160 --
  Min 0.08991382310123273
-- Generation 161 --
  Min 0.08991382310123273
-- Generation 162 --
  Min 0.08991382310123273
-- Generation 163 --
  Min 0.08991382310123273
-- Generation 164 --
  Min 0.08991382310123273
-- Generation 165 --
  Min 0.08991382310123273
-- Generation 166 --
  Min 0.08991382310123273
-- Generation 167 --
  Min 0.08991382310123273
-- Generation 168 --
  Min 0.08991382310123273
-- Generation 169 --
  Min 0.08991382310123273
-- Generation 170 --
  Min 0.08991382310123273
-- Generation 171 --
  Min 0.08991

  Min 0.08991382310123273
-- Generation 298 --
  Min 0.08991382310123273
-- Generation 299 --
  Min 0.08991382310123273
-- Generation 300 --
  Min 0.08991382310123273
[0.0907136110400153, 0.07686757918314185, 0.07607922787814222, 0.13088454655677279, 0.4749755878339671, 0.06083359177154894, 0.08964585573641165, 0.09129568476438957, 0.12138460625916153, 0.4743238549372174, 0.04753014535778734, 0.058225412473882146, 0.14829429458552223, 0.05894600162203982, 0.0551670654884726, 0.05244932247916685, 0.050276876179232946, 0.053533324768142584, 0.15642909283862705, 0.050248453257162484, 0.5818958649891957, 0.0575122713809732, 0.05368369687686615, 0.028101872232332438, 0.5480919596229314, 0.07007411316133574, 0.1753505301452422, 0.06718555658031836, 0.05688749126661802, 0.4737515261381581, 0.07437301975470292, 0.05697246909992869, 0.11161586410767352, 0.10833097231629404, 0.11806865731662478, 0.5023331261878421, 0.08886724373302707, 0.07369413298673674, 0.09987839647765792, 0.0790745500809617

  Min 0.08903856765319793
-- Generation 418 --
  Min 0.08903856765319793
-- Generation 419 --
  Min 0.08903856765319793
-- Generation 420 --
  Min 0.08903856765319793
-- Generation 421 --
  Min 0.08903856765319793
-- Generation 422 --
  Min 0.08903856765319793
-- Generation 423 --
  Min 0.08903856765319793
-- Generation 424 --
  Min 0.08903856765319793
-- Generation 425 --
  Min 0.08903856765319793
-- Generation 426 --
  Min 0.08903856765319793
-- Generation 427 --
  Min 0.08903856765319793
-- Generation 428 --
  Min 0.08903856765319793
-- Generation 429 --
  Min 0.08903856765319793
-- Generation 430 --
  Min 0.08903856765319793
-- Generation 431 --
  Min 0.08903856765319793
-- Generation 432 --
  Min 0.08903856765319793
-- Generation 433 --
  Min 0.08903856765319793
-- Generation 434 --
  Min 0.08903856765319793
-- Generation 435 --
  Min 0.08903856765319793
-- Generation 436 --
  Min 0.08903856765319793
-- Generation 437 --
  Min 0.08903856765319793
-- Generation 438 --
  Min 0.08903

  Min 0.08903835980692625
-- Generation 565 --
  Min 0.08903835980692625
-- Generation 566 --
  Min 0.08903835980692625
-- Generation 567 --
  Min 0.08903835980692625
-- Generation 568 --
  Min 0.08903835980692625
-- Generation 569 --
  Min 0.08903835980692625
-- Generation 570 --
  Min 0.08903835980692625
-- Generation 571 --
  Min 0.08903835980692625
-- Generation 572 --
  Min 0.08903835980692625
-- Generation 573 --
  Min 0.08903835980692625
-- Generation 574 --
  Min 0.08903835980692625
-- Generation 575 --
  Min 0.08903835980692625
-- Generation 576 --
  Min 0.08903835980692625
-- Generation 577 --
  Min 0.08903835980692625
-- Generation 578 --
  Min 0.08903835980692625
-- Generation 579 --
  Min 0.08903835980692625
-- Generation 580 --
  Min 0.08903835980692625
-- Generation 581 --
  Min 0.08903835980692625
-- Generation 582 --
  Min 0.08903835980692625
-- Generation 583 --
  Min 0.08903835980692625
-- Generation 584 --
  Min 0.08903835980692625
-- Generation 585 --
  Min 0.08903

...

  Min 0.08903835980692625
-- Generation 994 --
  Min 0.08903835980692625
-- Generation 995 --
  Min 0.08903835980692625
-- Generation 996 --
  Min 0.08903835980692625
-- Generation 997 --
  Min 0.08903835980692625
-- Generation 998 --
  Min 0.08903835980692625
-- Generation 999 --
  Min 0.08903835980692625
-- Generation 1000 --
  Min 0.08903835980692625
[0.0907136110400153, 0.0765802290922318, 0.07636657796905225, 0.10984466203234111, 0.4749755878339671, 0.06083359177154894, 0.11068574026084335, 0.09129568476438957, 0.12138460625916153, 0.4743238549372174, 0.04753014535778734, 0.058225412473882146, 0.14829429458552223, 0.05894600162203982, 0.10683071726243157, 0.05311649496997588, 0.05025806438601721, 0.05286615227733356, 0.10476544106466809, 0.05026726505037821, 0.5818958649891957, 0.05611677520454268, 0.05507919305329667, 0.028101872232332438, 0.5480919596229314, 0.07007411316133574, 0.1753505301452422, 0.06718555658031836, 0.05688749126661802, 0.4737515261381581, 0.09507272495973981

  Min 0.08903835980692625
-- Generation 1111 --
  Min 0.08903835980692625
-- Generation 1112 --
  Min 0.08903835980692625
-- Generation 1113 --
  Min 0.08903835980692625
-- Generation 1114 --
  Min 0.08903835980692625
-- Generation 1115 --
  Min 0.08903835980692625
-- Generation 1116 --
  Min 0.08903835980692625
-- Generation 1117 --
  Min 0.08903835980692625
-- Generation 1118 --
  Min 0.08903835980692625
-- Generation 1119 --
  Min 0.08903835980692625
-- Generation 1120 --
  Min 0.08903835980692625
-- Generation 1121 --
  Min 0.08903835980692625
-- Generation 1122 --
  Min 0.08903835980692625
-- Generation 1123 --
  Min 0.08903835980692625
-- Generation 1124 --
  Min 0.08903835980692625
-- Generation 1125 --
  Min 0.08903835980692625
-- Generation 1126 --
  Min 0.08903835980692625
-- Generation 1127 --
  Min 0.08903835980692625
-- Generation 1128 --
  Min 0.08903835980692625
-- Generation 1129 --
  Min 0.08903835980692625
-- Generation 1130 --
  Min 0.08903835980692625
-- Generation 

  Min 0.08903835980692625
-- Generation 1255 --
  Min 0.08903835980692625
-- Generation 1256 --
  Min 0.08903835980692625
-- Generation 1257 --
  Min 0.08903835980692625
-- Generation 1258 --
  Min 0.08903835980692625
-- Generation 1259 --
  Min 0.08903835980692625
-- Generation 1260 --
  Min 0.08903835980692625
-- Generation 1261 --
  Min 0.08903835980692625
-- Generation 1262 --
  Min 0.08903835980692625
-- Generation 1263 --
  Min 0.08903835980692625
-- Generation 1264 --
  Min 0.08903835980692625
-- Generation 1265 --
  Min 0.08903835980692625
-- Generation 1266 --
  Min 0.08903835980692625
-- Generation 1267 --
  Min 0.08903835980692625
-- Generation 1268 --
  Min 0.08903835980692625
-- Generation 1269 --
  Min 0.08903835980692625
-- Generation 1270 --
  Min 0.08903835980692625
-- Generation 1271 --
  Min 0.08903835980692625
-- Generation 1272 --
  Min 0.08903835980692625
-- Generation 1273 --
  Min 0.08903835980692625
-- Generation 1274 --
  Min 0.08903835980692625
-- Generation 

Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/alex/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/alex/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/alex/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/alex/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/alex/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/alex/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/alex/anaconda3/lib/python3.7/multiprocessing/queues.py", l

KeyboardInterrupt: 